In [2]:
!pip install neuron
!pip install mpi4py

In [3]:
!mpiexec --allow-run-as-root -n 4 python testmpi.py

--------------------------------------------------------------------------
There are not enough slots available in the system to satisfy the 4
slots that were requested by the application:

  python

Either request fewer slots for your application, or make more slots
available for use.

A "slot" is the Open MPI term for an allocatable unit where we can
launch a process.  The number of slots available are defined by the
environment in which Open MPI processes are run:

  1. Hostfile, via "slots=N" clauses (N defaults to number of
     processor cores if not provided)
  2. The --host command line parameter, via a ":N" suffix on the
     hostname (N defaults to 1 if not provided)
  3. Resource manager (e.g., SLURM, PBS/Torque, LSF, etc.)
  4. If none of a hostfile, the --host command line parameter, or an
     RM is present, Open MPI defaults to the number of processor cores

In all the above cases, if you want Open MPI to default to the number
of hardware threads instead of the number of

In [2]:
%%writefile ballandstick.py
from neuron import h
from neuron.units import ms, mV

class Cell:
    def __init__(self, gid, x, y, z, theta):
        self._gid = gid
        self._setup_morphology()
        self.all = self.soma.wholetree()
        self._setup_biophysics()
        self.x = self.y = self.z = 0
        h.define_shape()
        self._rotate_z(theta)
        self._set_position(x, y, z)

        # Set up a spike detector
        self._spike_detector = h.NetCon(self.soma(0.5)._ref_v, None, sec=self.soma)
        self.spike_times = h.Vector()
        self._spike_detector.record(self.spike_times)

        self._ncs = []  # to store NetCons
        self.soma_v = h.Vector().record(self.soma(0.5)._ref_v)

    def __repr__(self):
        return '{}[{}]'.format(self.name, self._gid)

    def _set_position(self, x, y, z):
        for sec in self.all:
            for i in range(sec.n3d()):
                sec.pt3dchange(i,
                               x - self.x + sec.x3d(i),
                               y - self.y + sec.y3d(i),
                               z - self.z + sec.z3d(i),
                               sec.diam3d(i))
        self.x, self.y, self.z = x, y, z

    def _rotate_z(self, theta):
        """Rotate the cell about the Z axis."""
        for sec in self.all:
            for i in range(sec.n3d()):
                x = sec.x3d(i)
                y = sec.y3d(i)
                c = h.cos(theta)
                s = h.sin(theta)
                xprime = x * c - y * s
                yprime = x * s + y * c
                sec.pt3dchange(i, xprime, yprime, sec.z3d(i), sec.diam3d(i))

class BallAndStick(Cell):
    name = 'BallAndStick'

    def _setup_morphology(self):
        self.soma = h.Section(name='soma', cell=self)
        self.dend = h.Section(name='dend', cell=self)
        self.dend.connect(self.soma)
        self.soma.L = self.soma.diam = 12.6157
        self.dend.L = 200
        self.dend.diam = 1

    def _setup_biophysics(self):
        for sec in self.all:
            sec.Ra = 100   # Axial resistance in Ohm*cm
            sec.cm = 1     # Membrane capacitance in microF/cm^2
        # Insert Hodgkin-Huxley channels in the soma:
        self.soma.insert('hh')
        for seg in self.soma:
            seg.hh.gnabar = 0.12
            seg.hh.gkbar = 0.036
            seg.hh.gl = 0.0003
            seg.hh.el = -54.3
        # Insert passive current in the dendrite:
        self.dend.insert('pas')
        for seg in self.dend:
            seg.pas.g = 0.001
            seg.pas.e = -65
        # Create a synapse at the middle of the dendrite:
        self.syn = h.ExpSyn(self.dend(0.5))
        self.syn.tau = 2 * ms


Overwriting ballandstick.py


In [3]:
%%writefile ring_net.py
from neuron import h
from ballandstick import BallAndStick

# Initialize MPI support before using ParallelContext.
h.nrnmpi_init()
pc = h.ParallelContext()

class Ring:
    """
    A network of N Ball-and-Stick cells arranged in a ring.
    Each cell n makes an excitatory synapse onto cell n+1,
    with the last cell connecting back to the first.
    """
    def __init__(self, N=5, stim_w=0.04, stim_t=9, stim_delay=1, syn_w=0.01, syn_delay=5, r=50):
        self._N = N
        self.set_gids()
        self._syn_w = syn_w
        self._syn_delay = syn_delay
        self._create_cells(r)
        self._connect_cells()
        # Only the process that owns cell with gid 0 gets the stimulus.
        if pc.gid_exists(0):
            self._netstim = h.NetStim()
            self._netstim.number = 1
            self._netstim.start = stim_t
            self._nc = h.NetCon(self._netstim, pc.gid2cell(0).syn)
            self._nc.delay = stim_delay
            self._nc.weight[0] = stim_w

    def set_gids(self):
        """Assign cell gids round-robin across available MPI hosts."""
        self.gidlist = list(range(pc.id(), self._N, pc.nhost()))
        for gid in self.gidlist:
            if not pc.gid_exists(gid):
                pc.set_gid2node(gid, pc.id())

    def _create_cells(self, r):
        self.cells = []
        for i in self.gidlist:
            theta = i * 2 * h.PI / self._N
            cell = BallAndStick(i, h.cos(theta) * r, h.sin(theta) * r, 0, theta)
            self.cells.append(cell)
        for cell in self.cells:
            pc.cell(cell._gid, cell._spike_detector)

    def _connect_cells(self):
        """Connect each cell's output synapse to the next cell's synapse in a ring."""
        for target in self.cells:
            source_gid = (target._gid - 1 + self._N) % self._N
            nc = pc.gid_connect(source_gid, target.syn)
            nc.weight[0] = self._syn_w
            nc.delay = self._syn_delay
            target._ncs.append(nc)


Overwriting ring_net.py


In [ ]:
from neuron import h
from neuron.units import ms, mV
h.load_file('stdrun.hoc')  # ensure the standard run library is loaded

from ring_net import Ring
import time

# Get the ParallelContext (initialized in ring_net.py)
from neuron import h
pc = h.ParallelContext()
pc.set_maxstep(10 * ms)  # Maximum time step for parallel communication

# Create the network first so that sections are defined
ring = Ring(N=5)

# Now record the simulation time vector
t_vec = h.Vector().record(h._ref_t)

# Set simulation duration
sim_time = 100 * ms

# Start the timer
start_time = time.time()

# Run the simulation using the parallel solver.
# (For a serial run in the notebook, you might use h.continuerun(sim_time) instead.)
pc.psolve(sim_time)

# End the timer
end_time = time.time()
execution_time = end_time - start_time

# Only the process owning cell with gid 0 prints the execution time
if pc.id() == 0:
    print("Total execution time: {:.3f} seconds".format(execution_time))

# Synchronize processes before finishing (important for parallel runs)
pc.barrier()
pc.done()
h.quit()


Total execution time: 0.012 seconds
